In [1]:
# !git clone https://github.com/guy-hadad/GenRec.git

In [2]:
%cd GenRec

/content/GenRec


In [3]:
# !pip install "datasets<4"

In [4]:
from data.utils import prepare_data_like_other_filtering
from tokenization.utils import build_asin_id_tokenizer

In [5]:
# ds_train, ds_valid, ds_test, asin2id = prepare_data("All_Beauty", K=0,  max_hist=50)
ds_train, ds_valid, ds_test, asin2id = prepare_data_like_other_filtering(
    name="All_Beauty",         # or your domain
    split_type="leave_one_out",         # or "leave_one_out"
    ratios=(0.8, 0.1, 0.1),
    max_hist=50,               # matches the second script's default
    filter_by_meta=True,
)
tokenizer = build_asin_id_tokenizer(asin2id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/9159 [00:00<?, ? examples/s]

Map:   0%|          | 0/9159 [00:00<?, ? examples/s]

Map:   0%|          | 0/9159 [00:00<?, ? examples/s]

In [6]:
def print_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Trainable ratio: {100 * trainable_params / total_params:.2f}%")


In [22]:
ds_train[0]

{'user_id': 'AFSKPY37N3C43SOI5IEXEK5JSIYA',
 'prompt': '15743 2307 23061 13243 37108 23283 27736 22928 19963 3257 4065 9840',
 'completion': '25712'}

In [43]:
# tiny_qwen3_addition_for_prompts.py
# Same training loop, now for datasets with {prompt, completion} and a prebuilt tokenizer.

import os, random
import numpy as np
import torch

from transformers import (
    Qwen3Config,
    Qwen3ForCausalLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    set_seed,
)

from datasets import DatasetDict

# -------------------------
# 0) Repro
# -------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); set_seed(SEED)

# --- helper: robust encode for str OR list (ints/strs) ---
def _encode_field(value, tok):
    # If it's already a string: standard encode
    if isinstance(value, str):
        return tok(value, add_special_tokens=False)

    # If it's a list (e.g., [123, 45, 6] or ["B08..", "B07.."]):
    # 1) cast ints -> strings
    # 2) tell HF these are pre-tokenized "words"
    if isinstance(value, list):
        tokens = [str(x) for x in value]
        return tok(tokens, add_special_tokens=False, is_split_into_words=True)

    # Anything else is unsupported
    raise ValueError(f"Unsupported field type for tokenization: {type(value)}")

# inverse map: id -> asin token
id2asin = {v: k for k, v in asin2id.items()}

def _to_token_words(value):
    """
    Convert value to a list of token strings that match tokenizer vocab.
    Your vocab has tokens like '36206', '8340', etc.
    """

    if isinstance(value, str):
        # "15743 2307 23061" -> ["15743", "2307", "23061"]
        return value.split()

    if isinstance(value, int):
        # 15743 -> ["15743"]
        return [str(value)]

    if isinstance(value, list):
        # [15743, 2307] or ["15743", "2307"] -> ["15743", "2307"]
        return [str(x) for x in value]

    raise ValueError(f"Unsupported field type for tokenization: {type(value)}")


# ensure left padding + special tokens exist
tokenizer.padding_side = "left"
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})
if tokenizer.eos_token_id is None:
    tokenizer.add_special_tokens({"eos_token": "<eos>"})

def tok_train(ex, tok):
    prompt_tokens     = _to_token_words(ex["prompt"])      # ["15743", "2307", ...]
    completion_tokens = _to_token_words(ex["completion"])  # ["25712"]

    eos = tok.eos_token_id

    enc = tok(prompt_tokens,     add_special_tokens=False, is_split_into_words=True)
    dec = tok(completion_tokens, add_special_tokens=False, is_split_into_words=True)

    input_ids = enc["input_ids"] + dec["input_ids"]  # + [eos]  # optional
    labels    = [-100] * len(enc["input_ids"]) + dec["input_ids"]

    return {
        "input_ids": input_ids,
        "attention_mask": [1] * len(input_ids),
        "labels": labels,
    }


def tok_eval(ex, tok):
    prompt_tokens     = _to_token_words(ex["prompt"])
    completion_tokens = _to_token_words(ex["completion"])

    eos = tok.eos_token_id

    enc = tok(prompt_tokens,     add_special_tokens=False, is_split_into_words=True)
    dec = tok(completion_tokens, add_special_tokens=False, is_split_into_words=True)

    input_ids = enc["input_ids"]
    labels    = dec["input_ids"]  # or dec["input_ids"] + [eos] if you want EOS in label

    return {
        "input_ids": input_ids,
        "attention_mask": [1] * len(input_ids),
        "labels": labels,
    }

# -------------------------
# 2) Collator (LEFT padding everywhere)
# -------------------------
def make_collate_fn(tokenizer):
    LABEL_PAD_ID = -100
    PAD_ID = tokenizer.pad_token_id

    def collate_fn(features):
        feats = [{
            "input_ids": list(f["input_ids"]),
            "attention_mask": list(f["attention_mask"]),
            "labels": list(f["labels"]),
        } for f in features]

        max_len = max(len(f["input_ids"]) for f in feats)

        batch_input_ids, batch_attn, batch_labels = [], [], []
        for f in feats:
            L_in, L_lb = len(f["input_ids"]), len(f["labels"])
            pad_in = max_len - L_in
            pad_lb = max_len - L_lb

            input_ids = [PAD_ID]*pad_in + f["input_ids"]
            attn      = [0]*pad_in     + f["attention_mask"]
            labels    = [LABEL_PAD_ID]*pad_lb + f["labels"]

            input_ids = input_ids[:max_len]
            attn      = attn[:max_len]
            labels    = labels[:max_len]

            batch_input_ids.append(input_ids)
            batch_attn.append(attn)
            batch_labels.append(labels)

        return {
            "input_ids": torch.tensor(batch_input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(batch_attn, dtype=torch.long),
            "labels": torch.tensor(batch_labels, dtype=torch.long),
        }
    return collate_fn







# -------------------------
# 4) Build tokenized datasets
# -------------------------
ds = DatasetDict({
    "train": ds_train,
    "validation": ds_valid,
    "test": ds_test,
})

tokenized_train = ds["train"].map(lambda ex: tok_train(ex, tokenizer),
                                  remove_columns=["prompt", "completion"])
tokenized_valid = ds["validation"].map(lambda ex: tok_eval(ex, tokenizer),
                                       remove_columns=["prompt", "completion"])
tokenized_test  = ds["test"].map(lambda ex: tok_eval(ex, tokenizer),
                                 remove_columns=["prompt", "completion"])

collate_fn = make_collate_fn(tokenizer)


def count_params(m):
    total = sum(p.numel() for p in m.parameters())
    trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
    print(f"Total parameters:     {total:,}  ({total/1e6:.2f}M)")
    print(f"Trainable parameters: {trainable:,}  ({trainable/1e6:.2f}M)")
# count_params(model)



Map:   0%|          | 0/9159 [00:00<?, ? examples/s]

Map:   0%|          | 0/9159 [00:00<?, ? examples/s]

Map:   0%|          | 0/9159 [00:00<?, ? examples/s]

In [54]:
# import numpy as np

# def make_compute_metrics(tokenizer=None):
#     LABEL_PAD_ID = -100

#     def compute_metrics(eval_pred):
#         preds, labels = eval_pred
#         labels = np.asarray(labels)
#         preds  = np.asarray(preds)
#         print(preds.shape)
#         print(labels.shape)
#         print(preds)
#         print(labels)

#         # Fallback: if logits slipped through, collapse to ids.
#         if preds.ndim == 3:  # (B, T, V)
#             preds = preds.argmax(-1)

#         total = 0
#         correct = 0

#         for p_row, l_row in zip(preds, labels):
#             l_row = np.asarray(l_row)
#             # first supervised label position (first non -100)
#             idxs = np.nonzero(l_row != LABEL_PAD_ID)[0]
#             if idxs.size == 0:
#                 continue
#             j = int(idxs[0])
#             if j == 0:  # no previous position to predict from
#                 continue

#             gold = int(l_row[j])
#             pred = int(p_row[j - 1])  # model predicts token j at position j-1
#             correct += int(pred == gold)
#             total  += 1

#         acc = (correct / total) if total else 0.0
#         return {"acc_first_token": acc, "n_examples": total}

#     return compute_metrics

# def preprocess_logits_for_metrics(logits, labels):
#     # Runs on each eval batch to reduce (B, T, V) -> (B, T) token ids
#     if isinstance(logits, tuple):
#         logits = logits[0]
#     return logits.argmax(dim=-1)

# compute_metrics = make_compute_metrics(tokenizer)

import numpy as np

def make_compute_metrics_recall_k(k=10):
    LABEL_PAD_ID = -100

    def compute_metrics(eval_pred):
        preds, labels = eval_pred
        # preds: (N, T, k) top-k token ids
        preds  = np.asarray(preds)
        labels = np.asarray(labels)

        # comment these out – they are huge
        # print(preds.shape)
        # print(labels.shape)
        # print(preds)
        # print(labels)

        total = 0
        hits  = 0

        for topk_row, l_row in zip(preds, labels):
            l_row = np.asarray(l_row)

            # find first supervised label index (first != -100)
            idxs = np.nonzero(l_row != LABEL_PAD_ID)[0]
            if idxs.size == 0:
                continue
            j = int(idxs[0])
            if j == 0:
                continue  # no previous position to predict from

            gold = int(l_row[j])          # gold token id
            cand_topk = topk_row[j - 1]   # shape (k,)

            if gold in cand_topk:
                hits += 1
            total += 1

        recall = hits / total if total else 0.0
        return {f"recall@{k}": recall, "n_examples": total}

    return compute_metrics

# compute_metrics = make_compute_metrics_recall_k(k=10)


def preprocess_logits_for_metrics(logits, labels):
    # logits: (B, T, V)
    import torch

    if isinstance(logits, tuple):
        logits = logits[0]

    # get top-k **indices** on GPU
    k = 10
    topk_indices = torch.topk(logits, k=k, dim=-1).indices  # (B, T, k)

    # Trainer will move this to CPU; no need for full logits
    return topk_indices


compute_metrics = make_compute_metrics_recall_k(k=10)

# scale = 512
# config = Qwen3Config(
#     vocab_size=tokenizer.vocab_size,
#     hidden_size=max(1, int(4096/scale)),
#     intermediate_size=max(1, int(22016/scale)),
#     num_hidden_layers=max(1, int(32/scale)),
#     num_attention_heads=max(1, int(32/scale)),
#     num_key_value_heads=max(1, int(32/scale)),
#     head_dim=max(1, int(128/scale)),
# )


config = Qwen3Config(
    vocab_size=tokenizer.vocab_size,
    hidden_size=64,
    intermediate_size=64,
    num_hidden_layers=2,
    num_attention_heads=2,
    num_key_value_heads=2,
    head_dim=16,
)


model = Qwen3ForCausalLM(config)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.generation_config.max_new_tokens = 1  # adjust if your completions are longer

def count_params(m):
    total = sum(p.numel() for p in m.parameters())
    trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
    print(f"Total parameters:     {total:,}  ({total/1e6:.2f}M)")
    print(f"Trainable parameters: {trainable:,}  ({trainable/1e6:.2f}M)")
count_params(model)

args = Seq2SeqTrainingArguments(
    output_dir="./qwen3_prompt_completion",
    eval_strategy="epoch",                  # <- correct arg name
    # eval_steps=0.1,  # int steps
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    per_device_train_batch_size=256,               # safer defaults; raise if you have GPU
    per_device_eval_batch_size=256,
    num_train_epochs=100,
    learning_rate=5e-3,
    warmup_steps=0,
    weight_decay=0.0,
    report_to="none",
    predict_with_generate=False,
        # generation_max_new_tokens=1,   # <-- use this, not model.generation_config
    generation_num_beams=1,        # deterministic
    include_inputs_for_metrics=False,
    fp16=False, bf16=False,
    seed=SEED,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    data_collator=collate_fn,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
        preprocess_logits_for_metrics=preprocess_logits_for_metrics,

)

trainer.train()
print("Validation:", trainer.evaluate())
print("Test:", trainer.evaluate(eval_dataset=tokenized_test))


/tmp/ipython-input-627710161.py:170: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 45277, 'bos_token_id': 45276, 'pad_token_id': 45275}.


Total parameters:     5,837,184  (5.84M)
Trainable parameters: 5,837,184  (5.84M)


Epoch,Training Loss,Validation Loss,Recall@10,N Examples
1,No log,10.508401,0.007424,9159
2,No log,11.290792,0.010154,9159
3,9.457800,11.563910,0.010481,9159
4,9.457800,11.896009,0.018015,9159
5,9.457800,11.876673,0.026968,9159
6,7.851900,12.365148,0.033519,9159
7,7.851900,12.416547,0.036467,9159
8,7.851900,12.773624,0.036248,9159
9,6.512900,13.123398,0.038214,9159
10,6.512900,13.577493,0.035921,9159


KeyboardInterrupt: 

In [ ]:
!nvidia-smi

In [ ]:
# scale = 32
# config = Qwen3Config(
#     vocab_size=tokenizer.vocab_size,
#     hidden_size=max(1, int(4096/scale)),
#     intermediate_size=max(1, int(22016/scale)),
#     num_hidden_layers=max(1, int(32/scale)),
#     num_attention_heads=max(1, int(32/scale)),
#     num_key_value_heads=max(1, int(32/scale)),
#     head_dim=max(1, int(128/scale)),
# )
# model = Qwen3ForCausalLM(config)
# model.config.pad_token_id = tokenizer.pad_token_id
# model.config.eos_token_id = tokenizer.eos_token_id
# model.generation_config.max_new_tokens = 1  # adjust if your completions are longer

# def count_params(m):
#     total = sum(p.numel() for p in m.parameters())
#     trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
#     print(f"Total parameters:     {total:,}  ({total/1e6:.2f}M)")
#     print(f"Trainable parameters: {trainable:,}  ({trainable/1e6:.2f}M)")
# count_params(model)

# # -------------------------
# # 6) Train
# # -------------------------
# args = Seq2SeqTrainingArguments(
#     output_dir="./qwen3_prompt_completion",
#     eval_strategy="steps",                  # <- correct arg name
#     eval_steps=0.1,  # int steps
#     logging_steps=100,
#     save_steps=1000,
#     save_total_limit=2,
#     per_device_train_batch_size=32,               # safer defaults; raise if you have GPU
#     per_device_eval_batch_size=64,
#     num_train_epochs=100,
#     learning_rate=5e-4,
#     warmup_steps=0,
#     weight_decay=0.0,
#     report_to="none",
#     predict_with_generate=True,
#     include_inputs_for_metrics=False,
#     fp16=False, bf16=False,
#     seed=SEED,
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=args,
#     tokenizer=tokenizer,
#     data_collator=collate_fn,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_valid,
#     compute_metrics=compute_metrics,
# )

# trainer.train()
# print("Validation:", trainer.evaluate())
# print("Test:", trainer.evaluate(eval_dataset=tokenized_test))


In [48]:
from transformers import AutoTokenizer

# assuming you already have your tokenizer and tokenized_train
sample = tokenized_valid[0]

# Decode the input_ids
decoded_input = tokenizer.decode(sample["input_ids"], skip_special_tokens=False)
print("Decoded input:", decoded_input)

# If you also have labels, you can decode them too
if "labels" in sample:
    decoded_labels = tokenizer.decode([t for t in sample["labels"] if t != -100], skip_special_tokens=False)
    print("Decoded labels:", decoded_labels)


Decoded input: 15743 2307 23061 13243 37108 23283 27736 22928 19963 3257 4065 9840 25712
Decoded labels: 19182


In [49]:
ds_valid[0]

{'user_id': 'AFSKPY37N3C43SOI5IEXEK5JSIYA',
 'prompt': '15743 2307 23061 13243 37108 23283 27736 22928 19963 3257 4065 9840 25712',
 'completion': '19182'}

In [ ]:
tokenizer.vocab